In [218]:
from ctypes import *
import time
from sys import path
import sys
from os import sep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

if sys.platform.startswith("win"):
    dwf = cdll.dwf
    constants_path = "C:" + sep + "Program Files (x86)" + sep + "Digilent" + sep + "WaveFormsSDK" + sep + "samples" + sep + "py"
elif sys.platform.startswith("darwin"):
    dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
    constants_path = "/Applications/WaveForms.app/Contents/Resources/SDK/samples/py"
else:
    dwf = cdll.LoadLibrary("libdwf.so")
    constants_path = "/usr/share/digilent/waveforms/samples/py"

path.append(constants_path)
import dwfconstants as constants

In [219]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0)) #setting VDD = 3V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +3.01V


In [220]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
WTIME_REG = 0x83
PERS_REG = 0x8C
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B
AILTL_REG = 0x84
AILTH_REG = 0x85
AIHTL_REG = 0x86
AIHTH_REG = 0x87



def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def clear_interrupt(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0


def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [221]:
write_register(ENABLE_REG, 0x01) #Power on / PON = 1
time.sleep(0.01)
write_register(ATIME_REG, 0xF6) #To run for 24ms, upto a max count of 10,240
write_register(CONTROL_REG, 0x02) #Setting AGAIN=16

write_register(PERS_REG, 0X00)
write_register(AILTL_REG, 0x00)
write_register(AILTH_REG, 0x00)
write_register(AIHTL_REG, 0x00)
write_register(AIHTH_REG, 0x00)

True

In [222]:
#Function to test the sensor output
def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [223]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)

In [217]:
cleanup()
#call it everytime after work is done

In [243]:
def Interrupt_check():

    #Setting the Interrupt values
    write_register(AILTH_REG,0x01)
    write_register(AILTL_REG,0xF4) #Setting Lower limit as 0x1F4 ~ 500
    write_register(AIHTH_REG,0x13)
    write_register(AIHTL_REG,0x88) #Setting Higher limit as 0x1388 ~ 5000
    print("Interrupt thresholds set")

    #Setting up the Persistence Register
    write_register(PERS_REG, 0b00000100) #Clear if 5 consecutive values are out of range

    #Enabling the AIEN and AEN
    write_register(ENABLE_REG, 0x13) #Now, also RGBC enable / PON = 1, AEN = 1, AIEN = 1

    c = read_colors()
    print(f"Clear channel level: {c['clear']}")

    print("\nSystem armed. Waiting for light level to rise above 5000...")
    print("\n Reading the Status Register")
    print("---"*5)
    STATUS_PIN = 0x93

    while True:
        status_val = read_register(STATUS_PIN)
        c = read_colors()
        if status_val & 0x10 != 0:
            print(f"Interrupt Triggered. Current Clear channel reading: {c['clear']}")
            if c['clear']<500: print(f">> Baseline ({c['clear']}) < Limit (500)")
            else: print(f">> Baseline ({c['clear']}) > Limit (5000)")
            time.sleep(0.05)
            rgClearCmd = (c_ubyte * 1)(0xE6)
            dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgClearCmd, c_int(1), byref(iNak))
            print("\nPlease wait, clearing the interrupt")
            for i in range(5):
                time.sleep(1)
                if i<5: print(".")
            print("Interrupt Cleared successfully!!")
            break
        else:
            print(f"Interrupt Not Triggered yet. Current clear Levels: {c['clear']}")
            #time.sleep(5)

    rgClearCmd = (c_ubyte * 1)(0xE6)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgClearCmd, c_int(1), byref(iNak))

    return None

In [246]:
Interrupt_check()

Interrupt thresholds set
Clear channel level: 3124

System armed. Waiting for light level to rise above 5000...

 Reading the Status Register
---------------
Interrupt Not Triggered yet. Current clear Levels: 3222
Interrupt Not Triggered yet. Current clear Levels: 3168
Interrupt Not Triggered yet. Current clear Levels: 3113
Interrupt Not Triggered yet. Current clear Levels: 3111
Interrupt Not Triggered yet. Current clear Levels: 3206
Interrupt Not Triggered yet. Current clear Levels: 3190
Interrupt Not Triggered yet. Current clear Levels: 3108
Interrupt Not Triggered yet. Current clear Levels: 3194
Interrupt Not Triggered yet. Current clear Levels: 3204
Interrupt Not Triggered yet. Current clear Levels: 3110
Interrupt Not Triggered yet. Current clear Levels: 3177
Interrupt Not Triggered yet. Current clear Levels: 3217
Interrupt Not Triggered yet. Current clear Levels: 3115
Interrupt Not Triggered yet. Current clear Levels: 3118
Interrupt Not Triggered yet. Current clear Levels: 3152
In